<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_4_Viernes_y_S%C3%A1bado_01_Nacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from zipfile import ZipFile
import requests
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from math import sin, cos, radians, sqrt
import matplotlib.pyplot as plt
import folium

In [ ]:
def download(url, directory):
    filename = url.split('/')[-1]
    filepath = os.path.join(directory, filename)

    # Verificar si el archivo ya existe
    if os.path.exists(filepath):
        print(f"El archivo {filename} ya existe, no se descarga de nuevo.")
        return

    # Proceso de descarga
    time.sleep(5)  # Simular retardo en la red
    r = requests.get(url, stream=True)
    total_size = int(r.headers['content-length'])
    with open(filepath, 'wb') as f:
        for data in tqdm(iterable=r.iter_content(chunk_size=1024), total=total_size / 1024, unit='KB'):
            f.write(data)

In [ ]:
def extract_shapefile(estados_geo, directory, shp_dir, shape_type):
    for estado in estados_geo:
        file = estado.split('_')[0]
        zip_file = os.path.join(directory, estado)
        shp_files = [
            f'conjunto_de_datos/{file}{shape_type}.shp',
            f'conjunto_de_datos/{file}{shape_type}.cpg',
            f'conjunto_de_datos/{file}{shape_type}.dbf',
            f'conjunto_de_datos/{file}{shape_type}.prj',
            f'conjunto_de_datos/{file}{shape_type}.shx'
        ]

        # Verificar si todos los archivos necesarios ya están descomprimidos
        if all(os.path.exists(os.path.join(shp_dir, f)) for f in shp_files):
            print(f"Todos los archivos para {file} ya están descomprimidos en {shp_dir}")
            continue

        # Proceso de descompresión
        with ZipFile(zip_file, 'r') as zip:
            for f in shp_files:
                try:
                    zip.extract(f, shp_dir)
                except KeyError:
                    if f.endswith('.cpg'):  # Archivo CPG es opcional
                        with open(os.path.join(shp_dir, f), 'w') as out_file:
                            out_file.write("ISO 88591")
                    else:
                        print(f"El archivo {f} no se encontró en el ZIP.")

In [ ]:
estados_geo = ["01_aguascalientes.zip","02_bajacalifornia.zip","03_bajacaliforniasur.zip","04_campeche.zip","05_coahuiladezaragoza.zip","06_colima.zip","07_chiapas.zip","08_chihuahua.zip","09_ciudaddemexico.zip","10_durango.zip","11_guanajuato.zip","12_guerrero.zip","13_hidalgo.zip","14_jalisco.zip","15_mexico.zip","16_michoacandeocampo.zip","17_morelos.zip","18_nayarit.zip","19_nuevoleon.zip","20_oaxaca.zip","21_puebla.zip","22_queretaro.zip","23_quintanaroo.zip","24_sanluispotosi.zip","25_sinaloa.zip","26_sonora.zip","27_tabasco.zip","28_tamaulipas.zip","29_tlaxcala.zip","30_veracruzignaciodelallave.zip","31_yucatan.zip","32_zacatecas.zip"]
estados_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13 ,14 ,15, 16, 17, 18, 19, 20, 21, 22, 23 ,24, 25, 26, 27, 28, 29, 30, 31, 32]

In [ ]:
download_directory = "./inegi/ccpvagebmza/"
csv_directory = download_directory + "csv/"

os.makedirs(download_directory, exist_ok=True)
os.makedirs(csv_directory, exist_ok=True)
os.makedirs("./inegi/mgccpv/gpkg", exist_ok=True)


In [ ]:
# Descargar datos estadísticos por estado
for i in estados_num:
    estado = str(i).zfill(2)
    zip_file_path = f'{download_directory}ageb_mza_urbana_{estado}_cpv2020_csv.zip'
    csv_file_path = f'{csv_directory}conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv'

    # Comprobar si el archivo CSV ya ha sido extraído
    if not os.path.exists(csv_file_path):
        url = f'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_{estado}_cpv2020_csv.zip'

        # Comprobar si el archivo ZIP ya existe antes de descargarlo
        if not os.path.exists(zip_file_path):
            download(url, download_directory)  # Asegúrate de que la función `download` acepte el directorio de destino
        # Descomprimir el archivo CSV específico si el ZIP ya está disponible
        with ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extract(f'ageb_mza_urbana_{estado}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv', csv_directory)
    else:
        print(f'El archivo CSV para el estado {estado} ya existe y no se descargará ni descomprimirá de nuevo.')

In [ ]:
url_mgccpv = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/"
for state in estados_geo:
    download(url_mgccpv + state, "./inegi/mgccpv")

# Manzanas
shape_type = "m"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(estados_geo,directory,shp_dir,shape_type)

In [ ]:
for estado in estados_geo:
    file = estado.split('_')[0]
    print("procesando estado: "+ file)
    gpdf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{file}m.shp")
    df = pd.read_csv(f"./inegi/ccpvagebmza/csv/ageb_mza_urbana_{file}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{file}_cpv2020.csv",na_values=['N/A','N/D','*'])
    df = df[["ENTIDAD", "MUN", "LOC", "AGEB", "MZA","POBTOT", "P_15A17", "P_18A24"]]
    df['CVEGEO'] = df.apply(lambda row: str(row['ENTIDAD']).zfill(2) + str(row['MUN']).zfill(3)+ str(row['LOC']).zfill(4)+ str(row['AGEB']).zfill(4)+ str(row['MZA']).zfill(3), axis=1)
    df_geo_censo = pd.merge(df, gpdf, how = 'left').drop(["CVE_ENT", "CVE_MUN", "CVE_LOC", "CVE_AGEB", "CVE_MZA"], axis = 1)
    df_geo_censo = df_geo_censo.drop(df[df.MZA == 0].index)
    df_geo_censo = gpd.GeoDataFrame(df_geo_censo, geometry="geometry")
    print("guardando datos del estado: "+str(estado))
    df_geo_censo.to_file(f"./inegi/mgccpv/gpkg/cpv2020_{file}.shp")

In [ ]:
dfs = []
for estado in estados_geo:
    file = estado.split('_')[0]
    gdf = gpd.read_file(f"./inegi/mgccpv/gpkg/cpv2020_{file}.shp")
    dfs.append(gdf)

censo_df = pd.concat(dfs, ignore_index=True)

In [ ]:
censo_df.loc[:, 'centroid'] = censo_df['geometry'].centroid

In [ ]:
censo_df_centroide = censo_df.set_geometry('centroid')

In [ ]:
censo_df_centroide.drop(columns=["geometry"], inplace=True)

In [ ]:
censo_df_centroide

In [ ]:
censo_df_centroide.to_file(f"./inegi/mgccpv/gpkg/cpv2020_centroides_INE.shp")